In [ ]:
from documents import Documents
import json


            
file_path=""
title=""
docs = Documents(file_path)

#Choose between 'recursive' 'fine_tuned_slm' and 'original' chunking
#Alternatively use llmsherpa at the bottom of the notebook

recursive_chunks=docs.add_file("recursive","test", title, file_path, file_path,chunk_size=1024)

with open("recursive_chunks.json", "w") as f:
    json.dump(recursive_chunks, f, indent=2)

fine_tuned_slm_chunks=docs.add_file("fine_tuned_slm","test", title, file_path, file_path,chunk_size=1024)

with open("fine_tuned_slm_chunks.json", "w") as f:
    json.dump(fine_tuned_slm_chunks, f, indent=2)

In [4]:
from ipytree import Tree, Node

def create_tree(data, name='root'):
    node = Node(name)
    if isinstance(data, dict):
        for key, value in data.items():
            node.add_node(create_tree(value, key))
    elif isinstance(data, list):
        for i, item in enumerate(data):
            node.add_node(create_tree(item, f'item_{i}'))
    else:
        node.name = f'{name}: {data}'
    return node

In [ ]:

tree = Tree(name="Recursive Chunking")
#tree.add_node(create_tree(chunks))
root_node = create_tree(recursive_chunks, "Recursive Chunking")
tree.add_node(root_node)
display(tree)

tree2 = Tree(name="Fine Tuned SLM Chunking")
#tree.add_node(create_tree(chunks))
root_node2 = create_tree(fine_tuned_slm_chunks, "Fine Tuned SLM Chunking")
tree2.add_node(root_node2)
display(tree2)

llmsherpa: https://github.com/nlmatics/llmsherpa?tab=readme-ov-file.  
The nlm-ingestor server is located here: https://github.com/nlmatics/nlm-ingestor 
Write up: https://ambikasukla.substack.com/p/efficient-rag-with-document-layout?r=ft8uc&utm_campaign=post&utm_medium=web&triedRedirect=true

In [ ]:
from llmsherpa.readers import LayoutPDFReader
from IPython.core.display import display, HTML
import os


llmsherpa_api_url = "http://localhost:5010/api/parseDocument"#?renderFormat=all&useNewIndentParser=true"
# pdf_url = "https://arxiv.org/pdf/1910.13461.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
# pdf_url = "https://www.apache.org/licenses/LICENSE-2.0.txt"
# pdf_url = "https://microsoft.gcs-web.com/static-files/931d7780-ccfc-47e3-97ad-09d87e12b795"
# pdf_url = "https://podcasts.ceu.edu/sites/podcasts.ceu.edu/files/sample.doc"
# pdf_url = "/Users/ambikasukla/projects/data/Kubernetes.docx"
#pdf_url = "https://en.wikipedia.org/wiki/Language_model"

file_path=""
#pdf_url = "https://github.com/nlmatics/nlm-ingestor"
# pdf_url = "https://raw.githubusercontent.com/nlmatics/llmsherpa/main/README.md"
# pdf_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=35362092&retmode=xml"
# pdf_url = "https://solutions.weblite.ca/pdfocrx/scansmpl.pdf"
do_ocr = True
#if do_ocr:
#    llmsherpa_api_url = llmsherpa_api_url + "&applyOcr=yes"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(file_path)

In [ ]:
sections = doc.sections()
section_list = [section.to_text() for section in sections]

with open("llmsherpa_chunks.json", "w") as f:
    json.dump(section_list, f, indent=2)

tree3 = Tree(name="LLMsherpa Sections")

root_node3 = create_tree(fine_tuned_slm_chunks, "LLMsherpa Sections")
tree3.add_node(root_node3)
display(tree3)


In [ ]:
from ipytree import Tree, Node

def create_tree(data, name='root'):
    if isinstance(data, dict):
        node = Node(name)
        for key, value in data.items():
            if key == 'chunks':
                for chunk in value:
                    chunk_node = Node(f"{chunk['title']}: {chunk['text'][:50]}...")
                    node.add_node(chunk_node)
            else:
                node.add_node(create_tree(value, key))
    elif isinstance(data, list):
        node = Node(name)
        for i, item in enumerate(data):
            node.add_node(create_tree(item, f'item_{i}'))
    else:
        node = Node(f'{name}: {data}')
    return node

# Assume 'doc' is your LLMSherpa document object
sections = doc.sections()

# Process sections and chunks
processed_data = {
    "Document": {
        section.title: {
            "chunks": [
                {"title": section.title, "text": chunk.to_text()}
                for chunk in section.chunks()
            ]
        }
        for section in sections
    }
}

# Create the tree
root_node = Node("Document Structure")
data_node = create_tree(processed_data, "Sections")
root_node.add_node(data_node)

# Create and display the tree
tree = Tree()
tree.add_node(root_node)
display(tree)